Exercise 2.77.

Let's analize `apply-generic` procedure below. It takes the combinations of options and args. Derive type-tag from args. The tag is used to direct to the appropriate package.

In the complex package in textbook, there is no `(put 'magnitude  '(complex))`, so the `apply-generic` can't `(get 'magnitude '(complex))`.

It's a two-level tag system. The outer tag (complex) is used to direct to the `magnitude` of the complex package. Within the complex package, the next tag (rectangular) is used to direct to `magnitude-rectangular` .

In [ ]:
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
            "No method for these types -- APPLY-GENERIC"
            (list op type-tags))))))

Exercise 2.78.

Original:

In [ ]:
(define (attach-tag type-tag contents)
  (cons type-tag contents))
(define (type-tag datum)
  (if (pair? datum)
      (car datum)
      (error "Bad tagged datum -- TYPE-TAG" datum)))
(define (contents datum)
  (if (pair? datum)
      (cdr datum)
      (error "Bad tagged datum -- CONTENTS" datum)))

Modified:

In [4]:
(define (attach-tag type-tag contents)
  (if (number? contents)
      contents
      (cons type-tag contents)))
  
(define (type-tag datum)
  (if (number? datum)
      'scheme-number
      (if (pair? datum)
          (car datum)
          (error "Bad tagged datum -- TYPE-TAG" datum))))

(define (contents datum)
  (if (number? datum)
      datum
      (if (pair? datum)
          (cdr datum)
          (error "Bad tagged datum -- CONTENTS" datum))))

contents

In [11]:
(eq? 't 't)

#t

Exercise 2.79.

In [6]:
;The generic arithmetic procedures
(define (equ? x y) (apply-generic 'equ? x y))

(define (install-equ?-package)
  (put 'equ? '(scheme-number scheme-number) eq?)
  
  (define (equ-rat x y)
    (and (eq? (numer x) (numer y)) (eq? (denom x) (denom y))))
  (put 'equ? '(rational rational) equ-rat)
  
  
  (define (equ-complex x y)
    (and (eq? (real-part x) (real-part y)) (eq? (imag-part x) (imag-part y))))
  (put 'equ? '(complex complex) equ-complex))

install-equ?-package

Exercise 2.80.

*Same as 2.79*

In [20]:
(define (=zero? x ) (apply-generic '=zero? x))

(define (install-=zero?-package)
  (put '=zero? '(scheme-number) (lambda (x) (= x 0)))
  
  (put '=zero? '(rational) (lambda (x) (=zero? (numer x))))
  
  (put '=zero? '(complex) =zero?)
  
  (put '=zero? '(rectangular) (and (=zero? (real-part x)) (=zero? (imag-part x))))
  (put '=zero? '(polar) (=zero? (magnitude x))))

install-=zero?-package

In [18]:
(= 1 1 1 1)

#t

Exercise 2.81.

a. That would be a infinite loop. `apply-generic` is a recursive function. "coerce" arguments of each type to their own type will feed the `apply-generic` procedure same arguments again and again.

b. Yes.

c.

In [21]:
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (if (equal? type1 type2)
                    (error "No method for these types"
                           (list op type-tags))
                    (let ((t1->t2 (get-coercion type1 type2))
                      (t2->t1 (get-coercion type2 type1)))
                      (cond (t1->t2
                             (apply-generic op (t1->t2 a1) a2))
                            (t2->t1
                             (apply-generic op a1 (t2->t1 a2)))
                            (else
                             (error "No method for these types"
                                    (list op type-tags)))))))
                
              (error "No method for these types"
                     (list op type-tags)))))))

apply-generic

Exercise 2.82.

The tower of types and a `raise` procedure is best solution. 

There is no need to implement this by coersion procedure. If have to, we can build it by Union-find, which based on the coersion procedures: transforms a type (child) to a target type (parent).

Suppose `(define args (list complex-number real-number integer-number))` , and we have `intege->real`, `real->complex`. Apply the strategy mentioned in the question, the third argument: `integer-number` won't be coerced to `complex`. The suitable `real->complex` operations will not be tried on it.)

Exercise 2.83.

integer, rational, real, complex.

In [ ]:
(define (raise-int x)
  (make-rational (content x) 1))
(define (raise-rational x)
  (make-real (content x))
(define (raise-real x)
  (make-complex (content x) 0))
(put 'raise 'integer raise-int)
(put 'raise 'rational raise-rational)
(put 'raise 'real raise-real)
(define (raise x) (apply-generic 'raise x))


Exercise 2.84.

In [3]:
(define tower (list 'integer 'rational 'real 'complex))
(define (higher-in-tower type1 type2 tower-list)
  (if (= type1 type2)
      (type1 type2))
  (< (length (memq type1 tower-list)) (length (memq type2 tower-list))))

(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (if (= type1 type2)
                    (error "No method for these types"
                           (list op type-tags))
                    (let ((level1 (length (memq type1 tower-list)))
                      (level2 (length (memq type2 tower-list))))
                      (if (< level1 level2)
                          (apply-generic op a1 (raise a2))
                          (apply-generic op (raise a1) a2)))))   

              (error "No method for these types"
                     (list op type-tags)))))))

apply-generic

In [6]:
(higher-in-tower 'real 'rational tower)

#t

Exercise 2.85.

In [ ]:
(define (project-rational x)
  (make-integer  (numer x)))
(define (project-real x)
  (make-rational (- x (remainder x))))
(define (project-complex x)
  (make-real (real x)))

(put 'project 'rational project-rational)
(put 'project 'real project-real)
(put 'project 'complex project-complex)
(define (project x) (apply-generic 'drop x))

(define (drop x)
  (let ((p (project x)))
    (if (= x p)
        (project p)
        x)))


In [4]:
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (drop (apply proc (map contents args)))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (if (= type1 type2)
                    (error "No method for these types"
                           (list op type-tags))
                    (let ((level1 (length (memq type1 tower-list)))
                      (level2 (length (memq type2 tower-list))))
                      (if (< level1 level2)
                          (apply-generic op a1 (raise a2))
                          (apply-generic op (raise a1) a2)))))   

              (error "No method for these types"
                     (list op type-tags)))))))

apply-generic

Exercise 2.86.

Suppose we want to handle complex numbers whose real parts, imaginary parts, magnitudes, and angles can be either ordinary numbers, rational numbers or other numbers.

We must replace all the primitive operations **$+,-,\times,/$, square, sin, cos, sqrt, atan** with generic operations.

In [9]:
(define (add-complex z1 z2)
  (make-from-real-imag (add (real-part z1) (real-part z2))
                       (add (imag-part z1) (imag-part z2))))
(define (sub-complex z1 z2)
  (make-from-real-imag (sub (real-part z1) (real-part z2))
                       (sub (imag-part z1) (imag-part z2))))
(define (mul-complex z1 z2)
  (make-from-mag-ang (mul (magnitude z1) (magnitude z2))
                     (add (angle z1) (angle z2))))
(define (div-complex z1 z2)
  (make-from-mag-ang (div (magnitude z1) (magnitude z2))
                     (sub (angle z1) (angle z2))))

div-complex

In [ ]:
(define (square x) (mul x x))
(define (sin-for-rational))
(define (cos-for-rational))
(put 'square '(rational) (lambda ))
(put 'sin '(rational) (lambda))
(put 'cos '(rational) (lambda))